<center>
    <h1>Identify Fruits from Color Palette</h1>
</center>

In [248]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from PIL import ImageColor
import pickle

import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

## Load and Preprocess Data

In [124]:
fa = r'C:\Users\623285\Downloads\train\freshapples_test.csv'
fb = r'C:\Users\623285\Downloads\train\freshbanana_test.csv'
fo = r'C:\Users\623285\Downloads\train\freshoranges_test.csv'
ra = r'C:\Users\623285\Downloads\train\rottenapples_test.csv'
rb = r'C:\Users\623285\Downloads\train\rottenbanana_test.csv'
ro = r'C:\Users\623285\Downloads\train\rottenoranges_test.csv'

In [125]:
fa = pd.read_csv(fa, index_col=[0])
fb = pd.read_csv(fb, index_col=[0])
fo = pd.read_csv(fo, index_col=[0])
ra = pd.read_csv(ra, index_col=[0])
rb = pd.read_csv(rb, index_col=[0])
ro = pd.read_csv(ro, index_col=[0])

df_list = [fa, fb, fo, ra, rb, ro]

In [126]:
df = pd.concat(df_list, ignore_index=True)
df.head()

,hex_code,total_pixels,type
0,#FFFFFF,77706,freshapples
1,#801E15,37024,freshapples
2,#AE241A,24356,freshapples
3,#000000,21729,freshapples
4,#B43937,13971,freshapples


In [127]:
for i in range(0, len(df)):
    df.hex_code[i] = list(ImageColor.getcolor(df.hex_code[i], "RGB"))
    
df.head()

C:\Users\623285\AppData\Local\Temp\ipykernel_28836\2884372044.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.hex_code[i] = list(ImageColor.getcolor(df.hex_code[i], "RGB"))


,hex_code,total_pixels,type
0,"[255, 255, 255]",77706,freshapples
1,"[128, 30, 21]",37024,freshapples
2,"[174, 36, 26]",24356,freshapples
3,"[0, 0, 0]",21729,freshapples
4,"[180, 57, 55]",13971,freshapples


In [243]:
data = pd.DataFrame(df['hex_code'].to_list(), columns=['r','g', 'b'])
data = pd.concat([df, data], axis=1) 
data.head()

,hex_code,total_pixels,type,r,g,b
0,"[255, 255, 255]",77706,freshapples,255,255,255
1,"[128, 30, 21]",37024,freshapples,128,30,21
2,"[174, 36, 26]",24356,freshapples,174,36,26
3,"[0, 0, 0]",21729,freshapples,0,0,0
4,"[180, 57, 55]",13971,freshapples,180,57,55


In [244]:
## replace label
data['type'] = data['type'].replace(['freshapples'],'Fresh Apple')
data['type'] = data['type'].replace(['freshbanana'],'Fresh Banana')
data['type'] = data['type'].replace(['freshoranges'],'Fresh Orange')
data['type'] = data['type'].replace(['rottenapples'],'Rotten Apple')
data['type'] = data['type'].replace(['rottenbanana'],'Rotten Banana')
data['type'] = data['type'].replace(['rottenoranges'],'Rotten Orange')

# data_short = data
# data_short['type'] = data_short['type'].replace(['freshapples'],'Fresh')
# data_short['type'] = data_short['type'].replace(['freshbanana'],'Fresh')
# data_short['type'] = data_short['type'].replace(['freshoranges'],'Fresh')
# data_short['type'] = data_short['type'].replace(['rottenapples'],'Rotten')
# data_short['type'] = data_short['type'].replace(['rottenbanana'],'Rotten')
# data_short['type'] = data_short['type'].replace(['rottenoranges'],'Rotten')

In [245]:
data.type.unique()

array(['Fresh Apple', 'Fresh Banana', 'Fresh Orange', 'Rotten Apple',
       'Rotten Banana', 'Rotten Orange'], dtype=object)

In [238]:
data_short.type.unique()

array(['Fresh', 'Rotten'], dtype=object)

## Train/Test Split

In [246]:
y = data['type'].to_frame()
X = data.drop(['type', 'hex_code'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = 42)

In [247]:
y_short = data_short['type'].to_frame()
X_short = data_short.drop(['type', 'hex_code'], axis=1)
x_train_short, x_test_short, y_train_short, y_test_short = train_test_split(X_short, y_short, test_size=0.2, stratify=y_short, random_state = 42)

## Train Models and Save to .pkl

In [289]:
results = pd.DataFrame([], columns = ['Model', 'Trained For', 'Accuracy'])
name = []
tfor = []
acc = []

In [290]:
mod_dt = DecisionTreeClassifier()
mod_dt.fit(x_train,y_train)
prediction=mod_dt.predict(x_test)
print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))
pickle.dump(mod_dt, open('dt.pkl', 'wb'))
name.append('Decision Tree')
tfor.append('Fresh/Rotten *fruit type*')
acc.append(metrics.accuracy_score(prediction,y_test))

The accuracy of the Decision Tree is 0.732


In [291]:
mod_dt = DecisionTreeClassifier()
mod_dt.fit(x_train_short,y_train_short)
prediction=mod_dt.predict(x_test_short)
print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test_short)))
pickle.dump(mod_dt, open('dt_short.pkl', 'wb'))
name.append('Decision Tree')
tfor.append('Fresh/Rotten')
acc.append(metrics.accuracy_score(prediction,y_test_short))

The accuracy of the Decision Tree is 0.841


In [292]:
mod_dt = GaussianNB()
mod_dt.fit(x_train,y_train)
prediction=mod_dt.predict(x_test)
print('The accuracy of the Guassian Naive Bayes is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))
pickle.dump(mod_dt, open('gnb.pkl', 'wb'))
name.append('Guassian Naive Bayes')
tfor.append('Fresh/Rotten *fruit type*')
acc.append(metrics.accuracy_score(prediction,y_test))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the Guassian Naive Bayes is 0.262


In [293]:
mod_dt = GaussianNB()
mod_dt.fit(x_train_short,y_train_short)
prediction=mod_dt.predict(x_test_short)
print('The accuracy of the Guassian Naive Bayes is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test_short)))
pickle.dump(mod_dt, open('gnb_short.pkl', 'wb'))
name.append('Guassian Naive Bayes')
tfor.append('Fresh/Rotten')
acc.append(metrics.accuracy_score(prediction,y_test_short))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the Guassian Naive Bayes is 0.594


In [294]:
mod_dt = KNeighborsClassifier()
mod_dt.fit(x_train,y_train)
prediction=mod_dt.predict(x_test)
print('The accuracy of the KNN is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))
pickle.dump(mod_dt, open('knn.pkl', 'wb'))
name.append('KNN')
tfor.append('Fresh/Rotten *fruit type*')
acc.append(metrics.accuracy_score(prediction,y_test))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neighbors\_classification.py:200: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


The accuracy of the KNN is 0.510


In [295]:
mod_dt = KNeighborsClassifier()
mod_dt.fit(x_train_short,y_train_short)
prediction=mod_dt.predict(x_test_short)
print('The accuracy of the KNN is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test_short)))
pickle.dump(mod_dt, open('knn_short.pkl', 'wb'))
name.append('KNN')
tfor.append('Fresh/Rotten')
acc.append(metrics.accuracy_score(prediction,y_test_short))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neighbors\_classification.py:200: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


The accuracy of the KNN is 0.729


In [296]:
mod_dt = LinearDiscriminantAnalysis()
mod_dt.fit(x_train,y_train)
prediction=mod_dt.predict(x_test)
print('The accuracy of the LDA is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))
pickle.dump(mod_dt, open('lda.pkl', 'wb'))
name.append('LDA')
tfor.append('Fresh/Rotten *fruit type*')
acc.append(metrics.accuracy_score(prediction,y_test))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the LDA is 0.360


In [297]:
mod_dt = LinearDiscriminantAnalysis()
mod_dt.fit(x_train_short,y_train_short)
prediction=mod_dt.predict(x_test_short)
print('The accuracy of the LDA is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test_short)))
pickle.dump(mod_dt, open('lda_short.pkl', 'wb'))
name.append('LDA')
tfor.append('Fresh/Rotten')
acc.append(metrics.accuracy_score(prediction,y_test_short))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the LDA is 0.657


In [298]:
mod_dt = QuadraticDiscriminantAnalysis()
mod_dt.fit(x_train,y_train)
prediction=mod_dt.predict(x_test)
print('The accuracy of the QDA is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))
pickle.dump(mod_dt, open('qda.pkl', 'wb'))
name.append('QDA')
tfor.append('Fresh/Rotten *fruit type*')
acc.append(metrics.accuracy_score(prediction,y_test))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the QDA is 0.418


In [299]:
mod_dt = QuadraticDiscriminantAnalysis()
mod_dt.fit(x_train_short,y_train_short)
prediction=mod_dt.predict(x_test_short)
print('The accuracy of the QDA is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test_short)))
pickle.dump(mod_dt, open('qda_short.pkl', 'wb'))
name.append('QDA')
tfor.append('Fresh/Rotten')
acc.append(metrics.accuracy_score(prediction,y_test_short))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the QDA is 0.691


In [300]:
mod_dt = LogisticRegression()
mod_dt.fit(x_train,y_train)
prediction=mod_dt.predict(x_test)
print('The accuracy of the Logistic Regression is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))
pickle.dump(mod_dt, open('lr.pkl', 'wb'))
name.append('Logistic Regression')
tfor.append('Fresh/Rotten *fruit type*')
acc.append(metrics.accuracy_score(prediction,y_test))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the Logistic Regression is 0.296


C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [301]:
mod_dt = LogisticRegression()
mod_dt.fit(x_train_short,y_train_short)
prediction=mod_dt.predict(x_test_short)
print('The accuracy of the Logistic Regression is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test_short)))
pickle.dump(mod_dt, open('lr_short.pkl', 'wb'))
name.append('Logistic Regression')
tfor.append('Fresh/Rotten')
acc.append(metrics.accuracy_score(prediction,y_test_short))

C:\Users\623285\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the Logistic Regression is 0.632


In [302]:
results['Model'] = name
results['Trained For'] = tfor
results['Accuracy'] = acc

results

,Model,Trained For,Accuracy
0,Decision Tree,Fresh/Rotten *fruit type*,0.731945
1,Decision Tree,Fresh/Rotten,0.840878
2,Guassian Naive Bayes,Fresh/Rotten *fruit type*,0.261922
3,Guassian Naive Bayes,Fresh/Rotten,0.593517
4,KNN,Fresh/Rotten *fruit type*,0.509909
5,KNN,Fresh/Rotten,0.729442
6,LDA,Fresh/Rotten *fruit type*,0.359756
7,LDA,Fresh/Rotten,0.656848
8,QDA,Fresh/Rotten *fruit type*,0.418040
9,QDA,Fresh/Rotten,0.691351


## Testing Model

In [99]:
from statistics import mode

def pix_fruitID(data, model_path):
    # load model
    loaded_model = pickle.load(open(model_path, 'rb'))
#     for i in range(0, len(data)):
#         data.hex_code[i] = list(ImageColor.getcolor(data.hex_code[i], "RGB"))
#     df = data.drop('hex_code', axis=1)
    
    return loaded_model.predict(df), loaded_model.predict_prob(df)